# Converting the dataset into our separated tables

### Import the general table for exploration

In [244]:
import pandas as pd
import numpy as np 
import json

In [245]:
df_movies = pd.read_csv('movies_metadata.csv')
df_credits = pd.read_csv('credits.csv')

C:\Users\Mariam Barakat\AppData\Local\Temp\ipykernel_15008\1436460036.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies_metadata.csv')


In [246]:
df_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [247]:
df_credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


### Functions

In [248]:
#a set that takes all the empty rows we need to remove
empty_rows=set()
def json_to_list(col , df_score):
    # if crew we will parse from a specific key 
    if col == 'crew':
        # will move through the entire dataframe
        for i in range(0, len(df_score)):
            film_director =""
            # change the row content into a list
            df_list_ = eval(df_score.loc[i,col])
            for j in range(0,len(df_list_)):
                # parse the parts with directors only 
                if df_list_[j]['job'] == 'Director':
                    film_director = df_list_[j]['name']
                    df_score.loc[i,col] = str(film_director)
                    # break because we will not need to search more
                    break
            if film_director == "":
                # if we are empty then add to the empty set 
                empty_rows.add(i)
                df_score.loc[i,col] = ''
    elif col == 'cast':
        # loop through all the dataframe 
        for i in range(0, len(df_score)):
            # this list will hold the parsed data 
            list_ = []
            # we will convert the string list of dictionaries, into a list of dictionary that we can access
            df_list_ = eval(df_score.loc[i,col])
            # we will loop through this list of dictionary to extract the needed data 
            for j in range(0, len(df_list_)):
                temp_ = df_list_[j]['name']
                list_.append(temp_.lower())
            # if there is nothing then we will add this to rows that need to be deleted 
            if len(df_list_) == 0:
                empty_rows.add(i)
            # we are only interested in the first 4 actors 
            selected = list_[0:4]
            #we add this to the row 
            df_score.loc[i,col] =  str(selected)
    else:
        for i in range(0, len(df_score)):
            #loop through the entire dataframe 
            list_ = []
            #we will take each content in the row and change it to a list of dictionaries to be able to parse
            try: df_list_ = eval(df_score.loc[i,col])
            except: df_list_ = []
            
            for j in range(0, len(df_list_)):
                # we will select the part of data we are interested in
                temp_ = df_list_[j]['name']
                list_.append(temp_.lower())
            # this will happen when our list is empty, thus we will need to add it to the rows to be deleted 
            if len(df_list_) == 0:
                empty_rows.add(i)
            df_score.loc[i,col] = str(list_)
   

### Genres table 

In [249]:
df_genres = pd.read_csv('movies_metadata.csv')
df_genres.head(2)

C:\Users\Mariam Barakat\AppData\Local\Temp\ipykernel_15008\685571360.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_genres = pd.read_csv('movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [250]:
df_genres.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [251]:
df_genres.drop(['adult', 'belongs_to_collection', 'imdb_id','budget','homepage'
       , 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'], axis = 1, inplace = True) 

In [252]:
df_genres.head()

,genres,id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357
4,"[{'id': 35, 'name': 'Comedy'}]",11862


In [253]:
json_to_list('genres', df_genres)
df_genres.head(3)

,genres,id
0,"['animation', 'comedy', 'family']",862
1,"['adventure', 'fantasy', 'family']",8844
2,"['romance', 'comedy']",15602


In [257]:
def multivalued(df ,cols):
    new_df = pd.DataFrame(columns = cols )
    for row in range(len(df)):
        for i in range(len(eval(df.iloc[row, 0]))):
            parsed = eval(df.iloc[row,0])[i]
            ID = df.iloc[row,1]
            s2 = pd.Series([ID , parsed ], index= cols)
            new_df = pd.concat([new_df , s2.to_frame().T], ignore_index=True)
    return new_df


In [258]:
df_genres = multivalued(df_genres ,["id", "genres"])

In [259]:
df_genres = df_genres.reset_index(drop=True)

In [260]:
df_genres.head(5)

,id,genres
0,862,animation
1,862,comedy
2,862,family
3,8844,adventure
4,8844,fantasy


In [261]:
df_genres.to_csv('Genres_table.csv')

### Spoken Languages

In [262]:
df_lang = pd.read_csv('movies_metadata.csv')
df_lang.head(2)

C:\Users\Mariam Barakat\AppData\Local\Temp\ipykernel_15008\2338217460.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lang = pd.read_csv('movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [263]:
df_lang.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [264]:
df_lang.drop(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
         'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'], axis = 1, inplace = True) 

In [265]:
df_lang.head(4)

,id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]"


In [266]:
json_to_list('spoken_languages', df_lang)


In [267]:
df_lang.head(2)

,id,spoken_languages
0,862,['english']
1,8844,"['english', 'français']"


In [268]:
def multivalued2(df ,cols):
    new_df = pd.DataFrame(columns = cols )
    for row in range(len(df)):
        for i in range(len(eval(df.loc[row, "spoken_languages"]))):
            parsed = eval(df.loc[row,"spoken_languages"])[i]
            ID = df.loc[row,"id"]
            s2 = pd.Series([ID , parsed ], index= cols)
            new_df = pd.concat([new_df , s2.to_frame().T], ignore_index=True)
    return new_df

In [269]:
df_lang_2 = multivalued2(df_lang, ["id", "spoken_languages"])
df_lang_2.head(3)

,id,spoken_languages
0,862,english
1,8844,english
2,8844,français


In [270]:
df_lang_2.to_csv('spoken_languages_table.csv')

### Casting table

In [320]:
df_casting = pd.read_csv('credits.csv')
df_casting.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [321]:
df_casting.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [322]:
df_casting.drop(['crew'], axis = 1, inplace = True) 

In [323]:
df_casting.head(3)

,cast,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",15602


In [328]:
def multivalued3(df, cols):
    new_df = pd.DataFrame(columns = cols)
    for i in range(len(df)):
        try: df_list = eval(df_casting.loc[i,"cast"])
        except: df_list =[]
        if len(df_list) > 3:
            l = 3
        else:
            l = len(df_list)
        for j in range(l):
            #print(df_list[j]['cast_id'])
            movieid = df_casting.loc[i,"id"]
            castid = df_list[j]['cast_id']
            character_name = df_list[j]['character']
            actor_name = df_list[j]['name']
            order = df_list[j]['order']

            s2 = pd.Series([movieid,castid,character_name,actor_name,order ], index= cols)
            new_df = pd.concat([new_df , s2.to_frame().T], ignore_index=True)
    return new_df


In [329]:
df_casting = multivalued3(df_casting, ["movieid", "castid", "character_name", "actor_name", "order"])

In [330]:
df_casting.head(5)

,movieid,castid,character_name,actor_name,order
0,862,14,Woody (voice),Tom Hanks,0
1,862,15,Buzz Lightyear (voice),Tim Allen,1
2,862,16,Mr. Potato Head (voice),Don Rickles,2
3,8844,1,Alan Parrish,Robin Williams,0
4,8844,8,Samuel Alan Parrish / Van Pelt,Jonathan Hyde,1


In [331]:
df_casting.to_csv('Cast_table.csv')

### Crew 

In [332]:
df_crew = pd.read_csv('credits.csv')
df_crew.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [333]:
df_crew.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [334]:
df_crew.drop(['cast'], axis = 1, inplace = True) 

In [335]:
df_crew.head(3)

,crew,id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [340]:
def multivalued4(df, col):
    new_df = pd.DataFrame(columns = col)
    for i in range(len(df)):
        try: df_list = eval(df_crew.loc[i,"crew"])
        except: df_list =[]
        if len(df_list) > 3:
            l = 3
        else:
            l = len(df_list)
        for j in range(l):
            #print(df_list[j]['cast_id'])
            movieid = df_crew.loc[i,"id"]
            crewid = df_list[j]['credit_id']
            department = df_list[j]['department']
            job = df_list[j]['job']
            name = df_list[j]['name']
            s2 = pd.Series([movieid,crewid, name ,department, job], index= col)
            new_df = pd.concat([new_df , s2.to_frame().T], ignore_index=True)
    return new_df

    

In [341]:
df_crew = multivalued4(df_crew, ["movieid", "crewid", "name", "department", "job"])

In [342]:
df_crew.head(4)

,movieid,crewid,name,department,job
0,862,52fe4284c3a36847f8024f49,John Lasseter,Directing,Director
1,862,52fe4284c3a36847f8024f4f,Joss Whedon,Writing,Screenplay
2,862,52fe4284c3a36847f8024f55,Andrew Stanton,Writing,Screenplay
3,8844,52fe44bfc3a36847f80a7cd1,Larry J. Franco,Production,Executive Producer


In [343]:
df_crew.to_csv('Crew_table.csv')